## Imports

In [1]:
import inspect
import os  # paths para os scripts
import sys

sys.path.append(os.path.join(os.path.dirname(sys.path[0]), "src"))
sys.path.append(os.path.join(os.path.dirname(sys.path[0]), "src", "data"))
sys.path.append(os.path.join(os.path.dirname(sys.path[0]), "src", "features"))
sys.path.append(os.path.join(os.path.dirname(sys.path[0]), "src", "models"))
sys.path.append(os.path.join(os.path.dirname(sys.path[0]), "src", "visualization"))
import context as c  # shotscuts para os paths

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


## Separação Treino e Teste

In [3]:
df = pd.read_csv(c.DATA_PROCESSED_FLD/'datasetFinal.csv')

In [4]:
X, y = df.drop(['tr_win'], axis=1), df['tr_win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)

X_train shape:  (97928, 24)
X_test shape:  (24482, 24)


## Otimização de Parâmetros

In [6]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

In [8]:
params = {
    'n_estimators': (100, 600),
    'max_depth': (1, 10)
}  

In [9]:
def randomForest_hyper_parms(n_estimators, max_depth):
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    model = RandomForestClassifier(n_estimators=n_estimators,
                                   max_depth=max_depth,
                                   max_features='sqrt',
                                   criterion='gini'
                                   )
    return np.mean(cross_val_score(model,X_train,y_train,cv=10,scoring='roc_auc'))

In [10]:
otimizacao = BayesianOptimization(f=randomForest_hyper_parms,pbounds = params,random_state=1)
otimizacao.maximize(n_iter=15, init_points=8, acq='ei')
parametros = otimizacao.max['params']

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | 0.8302    | 4.753     | 460.2     |
| 2         | 0.7918    | 1.001     | 251.2     |
| 3         | 0.8088    | 2.321     | 146.2     |
| 4         | 0.8083    | 2.676     | 272.8     |
| 5         | 0.8298    | 4.571     | 369.4     |
| 6         | 0.8301    | 4.773     | 442.6     |
| 7         | 0.8088    | 2.84      | 539.1     |
| 8         | 0.7926    | 1.246     | 435.2     |
| 9         | 0.8299    | 4.544     | 460.0     |
| 10        | 0.8371    | 5.884     | 448.2     |
| 11        | 0.855     | 9.691     | 454.4     |
| 12        | 0.8512    | 8.571     | 356.7     |
| 13        | 0.7913    | 1.0       | 352.7     |
| 14        | 0.855     | 9.706     | 362.4     |
| 15        | 0.8551    | 9.908     | 475.6     |
| 16        | 0.859     | 10.0      | 483.2     |
| 17        | 0.7928    | 1.79      | 480.9     |
| 18        | 0.8551    | 9.094     | 490.2     |
